# USCIS Credible Fear Spreadsheets [WIP]

## Prototype complete for first table in spreadsheet

In [ ]:
## TODO Data Verification 

In [1]:
from google.colab import drive

drive.mount("/content/drive/", force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fexperimentsandconfigs%20https%3a%2f%2fwww.googleapis.com%2fauth%2fphotos.native&response_type=code

Enter your authorization code:
4/1AX4XfWh8-cecCQTV61oAbmzze-jqLcrw7Sfu2jwWT-_btDQ3C6omTfq2Pq0
Mounted at /content/drive/


In [5]:
%cd drive/Shareddrives/Data\ Products\ Team/Products/Immigration\ Data\ Hub/DataRepo/data-repo-mvp

/content/drive/Shareddrives/Data Products Team/Products/Immigration Data Hub/DataRepo/data-repo-mvp


USCIS Credible Fear Spreadsheets: USCIS releases twice-monthly spreadsheets with data on credible and reasonable fear interviews conducted. There are a few challenges.

First, when they started releasing these spreadsheets, sometime last year, they included data going back to Jan. 2019. But every new spreadsheet goes back a little less far—the most recent ones released only go back to Sept. 2020. 

So it’s not as though you can just open the most recent spreadsheet and get the full picture—instead, I’ve been compiling all the data in a separate spreadsheet. This means that every time a new version is released, I have to transfer the new data to my own spreadsheet, and also go back and make updates to prior data, because past months’ data keeps being adjusted. 

Also, the data is released in two-week increments, but two-week increments aren’t a super useful time period, so I also compile my own tracking of monthly data. One note: USCIS hasn’t released new spreadsheets in a month, so I’m not sure whether they are going to continue consistently putting them out.

In [12]:
import requests
import pandas as pd
import numpy as np
from pathlib import Path

pd.set_option("max_columns", 100)
pd.set_option("display.max_colwidth", 0)

In [19]:
base_folder = Path('./data/raw_source_files/uscis-cfr-semi-monthly-reports/')

In [9]:
url = "https://www.uscis.gov/sites/default/files/document/reports"

In [10]:
def download_file(url_base, filename, out_folder):
    
    full_url = f"{url}/{filename}"
    
    response = requests.get(str(full_url))
    if response.status_code == 200:
        # Write content in pdf file
        output_folder = Path(out_folder)
        output_folder.mkdir(exist_ok=True, parents=True)
        outpath = output_folder / f"{filename}"
        pdf = open(str(outpath), "wb")
        pdf.write(response.content)
        pdf.close()
        print(full_url)
    else:
        print("File not found:", full_url)


In [13]:
# lets just try all months from 1-> 12 
for i in range(1, 12+1):
    month = str(i).zfill(2)
    download_file(
        url,
        f"Congressional-Semi-Monthly-Report-{month}-01-20-to-{month}-15-21.xlsx",
        base_folder,
    )

https://www.uscis.gov/sites/default/files/document/reports/Congressional-Semi-Monthly-Report-01-01-20-to-01-15-21.xlsx
https://www.uscis.gov/sites/default/files/document/reports/Congressional-Semi-Monthly-Report-02-01-20-to-02-15-21.xlsx
File not found: https://www.uscis.gov/sites/default/files/document/reports/Congressional-Semi-Monthly-Report-03-01-20-to-03-15-21.xlsx
https://www.uscis.gov/sites/default/files/document/reports/Congressional-Semi-Monthly-Report-04-01-20-to-04-15-21.xlsx
https://www.uscis.gov/sites/default/files/document/reports/Congressional-Semi-Monthly-Report-05-01-20-to-05-15-21.xlsx
https://www.uscis.gov/sites/default/files/document/reports/Congressional-Semi-Monthly-Report-06-01-20-to-06-15-21.xlsx
File not found: https://www.uscis.gov/sites/default/files/document/reports/Congressional-Semi-Monthly-Report-07-01-20-to-07-15-21.xlsx
https://www.uscis.gov/sites/default/files/document/reports/Congressional-Semi-Monthly-Report-08-01-20-to-08-15-21.xlsx
https://www.usci

In [20]:
test_file_path = base_folder / "Congressional-Semi-Monthly-Report-01-01-20-to-01-15-21.xlsx"

In [21]:
def parse_excel_sheet(file, sheet_name=0, threshold=5):
    """
    parses multiple tables from an excel sheet into multiple data frame objects. Returns [dfs, df_mds],
    where dfs is a list of data frames and df_mds their potential associated metadata


    source: https://stackoverflow.com/questions/43367805/pandas-read-excel-multiple-tables-on-the-same-sheet
    """

    xl = pd.ExcelFile(file)
    entire_sheet = xl.parse(sheet_name=sheet_name)

    # count the number of non-Nan cells in each row and then the change in that number between adjacent rows
    n_values = np.logical_not(entire_sheet.isnull()).sum(axis=1)
    n_values_deltas = n_values[1:] - n_values[:-1].values

    # define the beginnings and ends of tables using delta in n_values
    table_beginnings = n_values_deltas > threshold
    table_beginnings = table_beginnings[table_beginnings].index
    table_endings = n_values_deltas < -threshold
    table_endings = table_endings[table_endings].index
    if (
        len(table_beginnings) < len(table_endings)
        or len(table_beginnings) > len(table_endings) + 1
    ):
        raise BaseException("Could not detect equal number of beginnings and ends")

    # look for metadata before the beginnings of tables
    md_beginnings = []
    for start in table_beginnings:
        md_start = n_values.iloc[:start][n_values == 0].index[-1] + 1
        md_beginnings.append(md_start)

    # make data frames
    dfs = []
    df_mds = []
    for ind in range(len(table_beginnings)):
        start = table_beginnings[ind] + 1
        if ind < len(table_endings):
            stop = table_endings[ind]
        else:
            stop = entire_sheet.shape[0]
        df = xl.parse(sheet_name=sheet_name, skiprows=start, nrows=stop - start)
        dfs.append(df)

        md = xl.parse(
            sheet_name=sheet_name,
            skiprows=md_beginnings[ind],
            nrows=start - md_beginnings[ind] - 1,
        ).dropna(axis=1)
        df_mds.append(md)
    return dfs, df_mds


def extract_csmr_sub_table(
    file_path,
    table_number=0,
    zero_symbols=["-"],
    primary_column=None,
    secondary_column=None,
    check_column_names=True,
):

    """
    TODO

    """

    # Grab the date info in the name to use as a label
    file_path_time_label = file_path.name.split("Congressional-Semi-Monthly-Report-")[
        1
    ].split("-to")[0]

    # Attempt to process this data
    #     try:

    # Use this function to segment the tables in the sheet
    test = parse_excel_sheet(file_path)

    # Grab the dataframes
    dfs = test[0]
    # Grab the table based on sequence number
    # TODO rename to generic name
    sub_table = dfs[table_number].copy()

    # Combine the columns data with the first row of data (which also contains column info) together
    cols = list(zip(sub_table.columns, sub_table.loc[0, :]))

    # Create clean column labels
    formatted_cols = []
    for col in cols:
        if isinstance(col[0], str):
            part1 = col[0]
            part2 = "" if pd.isnull(col[1]) else f"-{col[1]}"
            formatted_cols.append(f"{part1}{part2}")
        else:
            formatted_cols.append(f'{col[0].strftime("%m/%Y")}')

    # Assign the columns to the dataframe
    sub_table.columns = formatted_cols

    # Drop that first row since it is not data
    sub_table = sub_table.drop(0, axis=0)

    for symbol in zero_symbols:
        sub_table = sub_table.replace("-", 0)

    if check_column_names:
        print("Columns --> ", sub_table.columns)
        display(sub_table.head(8))
        primary_column = input("What is the primary column name ? ")
        secondary_column = input(
            "What is the secondary column name ? [Leave blank if none.] "
        )
        print(
            "Selected -- Primary: ",
            primary_column,
            "Selected Secondary: ",
            secondary_column,
        )

    # TODO assert something somewhere about the need for column names
    if secondary_column != "" and secondary_column is not None:
        sub_table[primary_column] = sub_table[primary_column].ffill()
        sub_table.insert(
            0,
            primary_column + "-" + secondary_column,
            sub_table[primary_column] + "-" + sub_table[secondary_column],
        )

        sub_table = sub_table.drop([primary_column, secondary_column], axis=1)
        primary_column = primary_column + "-" + secondary_column

    # The data is presented bi-monthly - we want to sum
    # it up to get monthly totals. The dataframe has duplicate columns
    # names which represent the first and second half of the month
    # Heere we sum up the duplicate columns to get monthly total
    sub_table = sub_table.groupby(level=0, axis=1).sum()

    # Add a flag so we know what timeframe this data covers
    sub_table["time_label"] = file_path_time_label

    # Build a lis of columns time ordered
    time_cols = []
    for i in sub_table.columns:
        try:
            time_cols.append((i, pd.to_datetime(i)))
        except:
            continue

    properly_sorted_date_cols = sorted(
        time_cols,
        key=lambda x: x[1],
    )

    # Reorder the dataframe and get labels up front and then data time ordered
    sub_table = sub_table[
        [primary_column, "time_label"] + [i[0] for i in properly_sorted_date_cols]
    ]
    print("Likley Successfull Extracting Data - Check Results")
    return sub_table


In [16]:
# test = parse_excel_sheet(file_path)

# Single Table Examples 

Lets grab the first table in the first sheet **All Credible Fear Cases**

In python things are "zero indexed" meaning that the first element when counting is considered element 0. So belove we pass 0 to the table_number argument to grab the first table. 

When we run this code it will ask us to input a primary and secondary column. The primary column is the first column with menaingful information. 

The secondary column is the second column with meaningful information. Not all tables will have a secondary column but you must have primary column. This is the column we use to label the rows of data. When there is a primary and secondary it generally means we have some sort of nested/merged field situation for the rows. 

The second table in the spreadsheet **Credible Fear Cases Subject to Third Country Transit Bar - Interim Final Rule2** is an example of this. 

So for this first example the primary column is "From-To", so type of past "From-To" in for primary column, and leave secondary column blank (just press enter)

**Example 1**

In [22]:
example1a = extract_csmr_sub_table(test_file_path, table_number=0)

Columns -->  Index(['Unnamed: 0', 'From-To', 'Unnamed: 2', '01/2020', '01/2020', '02/2020',
       '02/2020', '03/2020', '03/2020', '04/2020', '04/2020', '05/2020',
       '05/2020', '06/2020', '06/2020', '07/2020', '07/2020', '08/2020',
       '08/2020', '09/2020', '09/2020', '10/2020', '10/2020', '11/2020',
       '11/2020', '12/2020', '12/2020', '01/2021'],
      dtype='object')


,Unnamed: 0,From-To,Unnamed: 2,01/2020,01/2020,02/2020,02/2020,03/2020,03/2020,04/2020,04/2020,05/2020,05/2020,06/2020,06/2020,07/2020,07/2020,08/2020,08/2020,09/2020,09/2020,10/2020,10/2020,11/2020,11/2020,12/2020,12/2020,01/2021
1,NaN,Case Receipts,NaN,1877,2901,2097,2532,2055,1578,433,253,256,205,283,367,399,325,279,336,308,417,341,368,284,381,459,562,435
2,NaN,All Decisions,NaN,1316,2373,2419,2121,1866,2644,1002,704,467,214,251,313,240,470,412,276,239,259,499,446,242,335,410,388,571
3,NaN,Fear Established (Y),NaN,408,800,870,622,523,916,463,296,179,97,133,138,122,258,253,163,114,111,278,242,137,210,249,243,351
4,NaN,Fear Not Established (N),NaN,655,1236,1198,1199,992,1359,464,350,221,89,90,148,83,137,128,71,84,94,182,125,66,91,92,83,145
5,NaN,Administratively Closed,NaN,253,337,351,300,351,369,75,58,67,28,28,27,35,75,31,42,41,54,39,79,39,34,69,62,75


What is the primary column name ? From-To
What is the secondary column name ? [Leave blank if none.] 
Selected -- Primary:  From-To Selected Secondary:  
Likley Successfull Extracting Data - Check Results


In [23]:
example1a

,From-To,time_label,01/2020,02/2020,03/2020,04/2020,05/2020,06/2020,07/2020,08/2020,09/2020,10/2020,11/2020,12/2020,01/2021
1,Case Receipts,01-01-20,4778,4629,3633,686,461,650,724,615,725,709,665,1021,435
2,All Decisions,01-01-20,3689,4540,4510,1706,681,564,710,688,498,945,577,798,571
3,Fear Established (Y),01-01-20,1208,1492,1439,759,276,271,380,416,225,520,347,492,351
4,Fear Not Established (N),01-01-20,1891,2397,2351,814,310,238,220,199,178,307,157,175,145
5,Administratively Closed,01-01-20,590,651,720,133,95,55,110,73,95,118,73,131,75


Above we see the extracted data - comparing to the spreadsheet we can see we have successfully extracted the data. 

Now that we know the column information for this table we don't have to enter it manually. We can run the code and pass that information. See below.

In [ ]:
example1b = extract_csmr_sub_table(
    test_file_path, primary_column="From-To", check_column_names=False, table_number=0
)

<IPython.core.display.Javascript object>

Likley Successfull Extracting Data - Check Results


<IPython.core.display.Javascript object>

In [ ]:
example1b

,From-To,time_label,01/2020,02/2020,03/2020,04/2020,05/2020,06/2020,07/2020,08/2020,09/2020,10/2020,11/2020,12/2020,01/2021
1,Case Receipts,01-01-20,4778,4629,3633,686,461,650,724,615,725,709,665,1021,435
2,All Decisions,01-01-20,3689,4540,4510,1706,681,564,710,688,498,945,577,798,571
3,Fear Established (Y),01-01-20,1208,1492,1439,759,276,271,380,416,225,520,347,492,351
4,Fear Not Established (N),01-01-20,1891,2397,2351,814,310,238,220,199,178,307,157,175,145
5,Administratively Closed,01-01-20,590,651,720,133,95,55,110,73,95,118,73,131,75


<IPython.core.display.Javascript object>


Lets look at another example. 

**Example 2**

Here we have 2 columns with information, `From-To` which has a detailed info row label on 

"Subject to bar ... ", and then an unnamed column (`Unnamed: 2`) with the subset details on the types of decisions. 

In [ ]:
example2a = extract_csmr_sub_table(test_file_path, table_number=1)

<IPython.core.display.Javascript object>

Columns -->  Index(['Unnamed: 0', 'From-To', 'Unnamed: 2', '01/2020', '01/2020', '02/2020',
       '02/2020', '03/2020', '03/2020', '04/2020', '04/2020', '05/2020',
       '05/2020', '06/2020', '06/2020', '07/2020', '07/2020', '08/2020',
       '08/2020', '09/2020', '09/2020', '10/2020', '10/2020', '11/2020',
       '11/2020', '12/2020', '12/2020', '01/2021'],
      dtype='object')


,Unnamed: 0,From-To,Unnamed: 2,01/2020,01/2020,02/2020,02/2020,03/2020,03/2020,04/2020,04/2020,05/2020,05/2020,06/2020,06/2020,07/2020,07/2020,08/2020,08/2020,09/2020,09/2020,10/2020,10/2020,11/2020,11/2020,12/2020,12/2020,01/2021
1,NaN,Subject to Bar under 8 CFR 208.13(c)(4) - No Exception Established,Case Receipts,1018,1893,1199,1384,1156,984,307,195,175,132,137,55,0,0,0,0,0,0,0,0,0,0,0,0,0
2,NaN,NaN,All Decisions,647,1479,1566,1197,880,1749,802,558,337,166,181,179,3,1,0,0,0,0,0,0,0,0,0,0,0
3,NaN,NaN,Fear Established (Y),198,469,629,425,246,594,371,238,136,81,92,87,3,1,0,0,0,0,0,0,0,0,0,0,0
4,NaN,NaN,Fear Not Established (N),332,794,728,607,454,919,384,292,179,69,71,84,0,0,0,0,0,0,0,0,0,0,0,0,0
5,NaN,NaN,Administratively Closed,117,216,209,165,180,236,47,28,22,16,18,8,0,0,0,0,0,0,0,0,0,0,0,0,0
6,NaN,Subject to Bar under 8 CFR 208.13(c)(4) - Exception Established,Case Receipts,20,29,28,30,34,40,14,11,8,5,15,1,0,0,0,0,0,0,0,0,0,0,0,0,0
7,NaN,NaN,All Decisions,12,23,21,16,18,43,38,30,22,3,19,6,0,0,0,0,0,0,0,0,0,0,0,0,0
8,NaN,NaN,Fear Established (Y),11,20,20,12,13,36,34,22,19,1,17,6,0,0,0,0,0,0,0,0,0,0,0,0,0


What is the primary column name ? From-To
What is the secondary column name ? [Leave blank if none.] Unnamed: 2
Selected -- Primary:  From-To Selected Secondary:  Unnamed: 2
Likley Successfull Extracting Data - Check Results


<IPython.core.display.Javascript object>

In [ ]:
example2a

,From-To-Unnamed: 2,time_label,01/2020,02/2020,03/2020,04/2020,05/2020,06/2020,07/2020,08/2020,09/2020,10/2020,11/2020,12/2020,01/2021
1,Subject to Bar under 8 CFR 208.13(c)(4) - No Exception Established-Case Receipts,01-01-20,2911,2583,2140,502,307,192,0,0,0,0,0,0,0
2,Subject to Bar under 8 CFR 208.13(c)(4) - No Exception Established-All Decisions,01-01-20,2126,2763,2629,1360,503,360,4,0,0,0,0,0,0
3,Subject to Bar under 8 CFR 208.13(c)(4) - No Exception Established-Fear Established (Y),01-01-20,667,1054,840,609,217,179,4,0,0,0,0,0,0
4,Subject to Bar under 8 CFR 208.13(c)(4) - No Exception Established-Fear Not Established (N),01-01-20,1126,1335,1373,676,248,155,0,0,0,0,0,0,0
5,Subject to Bar under 8 CFR 208.13(c)(4) - No Exception Established-Administratively Closed,01-01-20,333,374,416,75,38,26,0,0,0,0,0,0,0
6,Subject to Bar under 8 CFR 208.13(c)(4) - Exception Established-Case Receipts,01-01-20,49,58,74,25,13,16,0,0,0,0,0,0,0
7,Subject to Bar under 8 CFR 208.13(c)(4) - Exception Established-All Decisions,01-01-20,35,37,61,68,25,25,0,0,0,0,0,0,0
8,Subject to Bar under 8 CFR 208.13(c)(4) - Exception Established-Fear Established (Y),01-01-20,31,32,49,56,20,23,0,0,0,0,0,0,0
9,Subject to Bar under 8 CFR 208.13(c)(4) - Exception Established-Fear Not Established (N),01-01-20,4,5,12,11,5,2,0,0,0,0,0,0,0
10,Subject to Bar under 8 CFR 208.13(c)(4) - Exception Established-Administratively Closed,01-01-20,0,0,0,1,0,0,0,0,0,0,0,0,0


<IPython.core.display.Javascript object>

# Extracting and combining data from many sheets

In [24]:
def attempt_to_collect_and_combine_all_subsheets(
    data_folder, table_number, primary_column, secondary_column=""
):

    all_available_subsheets = pd.DataFrame()
    for fp in Path(data_folder).iterdir():
        if fp.name.endswith("xlsx") and "$" not in fp.name:
            print("\n", fp)
            try:
                all_available_subsheets = all_available_subsheets.append(
                    extract_csmr_sub_table(
                        fp,
                        table_number=table_number,
                        primary_column=primary_column,
                        secondary_column=secondary_column,
                        check_column_names=False,
                    )
                )
            except BaseException as e:
                print(f"ERROR - {e}")
    return all_available_subsheets.sort_values("time_label")

In [25]:
all_cfr = attempt_to_collect_and_combine_all_subsheets(
    base_folder, 0, primary_column="From-To"
)


 data/raw_source_files/uscis-cfr-semi-monthly-reports/Congressional-Semi-Monthly-Report-01-01-20-to-01-15-21.xlsx
Likley Successfull Extracting Data - Check Results

 data/raw_source_files/uscis-cfr-semi-monthly-reports/Congressional-Semi-Monthly-Report-02-01-20-to-02-15-21.xlsx
Likley Successfull Extracting Data - Check Results

 data/raw_source_files/uscis-cfr-semi-monthly-reports/Congressional-Semi-Monthly-Report-06-01-20-to-06-15-21.xlsx
Likley Successfull Extracting Data - Check Results

 data/raw_source_files/uscis-cfr-semi-monthly-reports/Congressional-Semi-Monthly-Report-04-01-20-to-04-15-21.xlsx
Likley Successfull Extracting Data - Check Results

 data/raw_source_files/uscis-cfr-semi-monthly-reports/Congressional-Semi-Monthly-Report-05-01-20-to-05-15-21.xlsx
Likley Successfull Extracting Data - Check Results

 data/raw_source_files/uscis-cfr-semi-monthly-reports/Congressional-Semi-Monthly-Report-10-01-20-to-10-15-21.xlsx
Likley Successfull Extracting Data - Check Results

 da

Above we see the print out of the files we have attempted to process - We see that there was an error processing the August file (EC-MD: TODO try to figure out why).

But there rest of the files were processed without obvious error. 

Lets take a look at the output. Running the cell below will show the output. We see we have each of the 5 different decision categories for each time label (taken from the file name). We also have columns for every time period encounted in the files. 



In [26]:
all_cfr

,From-To,time_label,01/2020,02/2020,03/2020,04/2020,05/2020,06/2020,07/2020,08/2020,09/2020,10/2020,11/2020,12/2020,01/2021,02/2021,03/2021,04/2021,05/2021,06/2021,07/2021,08/2021,09/2021,10/2021
1,Case Receipts,01-01-20,4778.0,4629.0,3633.0,686.0,461.0,650.0,724.0,615.0,725.0,709,665,1021,435,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,All Decisions,01-01-20,3689.0,4540.0,4510.0,1706.0,681.0,564.0,710.0,688.0,498.0,945,577,798,571,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Fear Established (Y),01-01-20,1208.0,1492.0,1439.0,759.0,276.0,271.0,380.0,416.0,225.0,520,347,492,351,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Fear Not Established (N),01-01-20,1891.0,2397.0,2351.0,814.0,310.0,238.0,220.0,199.0,178.0,307,157,175,145,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Administratively Closed,01-01-20,590.0,651.0,720.0,133.0,95.0,55.0,110.0,73.0,95.0,118,73,131,75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Case Receipts,02-01-20,NaN,4629.0,3632.0,686.0,461.0,650.0,724.0,615.0,725.0,709,665,1021,1279,1174.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,All Decisions,02-01-20,NaN,4540.0,4510.0,1706.0,681.0,564.0,710.0,687.0,498.0,945,578,798,1047,888.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Fear Established (Y),02-01-20,NaN,1492.0,1442.0,759.0,276.0,271.0,381.0,417.0,225.0,520,352,495,634,540.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Fear Not Established (N),02-01-20,NaN,2397.0,2348.0,814.0,310.0,238.0,219.0,197.0,178.0,307,153,172,287,244.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Administratively Closed,02-01-20,NaN,651.0,720.0,133.0,95.0,55.0,110.0,73.0,95.0,118,73,131,126,104.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Create Clean Summary Table**

Now that we have all this data we want to create a summary table that just has the most recent data for each category

In [27]:
def create_final_summary(df, column_label):

    compiled_rows = []

    info_elems = df[column_label].unique()

    # Build a lis of columns time ordered
    time_cols = []
    for i in df.columns:
        try:
            time_cols.append((i, pd.to_datetime(i)))
        except:
            continue

    properly_sorted_date_cols = sorted(
        time_cols,
        key=lambda x: x[1],
    )

    for elem in info_elems:
        tmp = df[df[column_label] == elem]
        compiled_rows.append(
            tmp[[i[0] for i in properly_sorted_date_cols]].ffill().iloc[-1]
        )

    out = pd.concat(compiled_rows, axis=1)

    out.columns = info_elems

    out = out.T.reset_index(drop=False)
    out = out.rename(columns={"index": column_label})
    return out

In [28]:
create_final_summary(all_cfr, "From-To")

,From-To,01/2020,02/2020,03/2020,04/2020,05/2020,06/2020,07/2020,08/2020,09/2020,10/2020,11/2020,12/2020,01/2021,02/2021,03/2021,04/2021,05/2021,06/2021,07/2021,08/2021,09/2021,10/2021
0,Case Receipts,4778.0,4629.0,3632.0,686.0,461.0,651.0,724.0,612.0,725.0,709.0,664.0,1020.0,1279.0,2503.0,5234.0,6611.0,7440.0,10104.0,8535.0,8707.0,6349.0,3276.0
1,All Decisions,3689.0,4540.0,4510.0,1706.0,681.0,564.0,710.0,689.0,498.0,945.0,578.0,799.0,1047.0,1631.0,2884.0,3293.0,4466.0,6206.0,7616.0,7072.0,7428.0,3184.0
2,Fear Established (Y),1208.0,1492.0,1442.0,760.0,276.0,275.0,382.0,418.0,225.0,522.0,352.0,496.0,639.0,1043.0,1856.0,2199.0,3047.0,4399.0,5271.0,5285.0,4881.0,2078.0
3,Fear Not Established (N),1891.0,2397.0,2348.0,813.0,310.0,234.0,218.0,198.0,178.0,305.0,153.0,172.0,281.0,432.0,746.0,915.0,1218.0,1623.0,1950.0,1722.0,2497.0,1112.0
4,Administratively Closed,590.0,651.0,720.0,133.0,95.0,55.0,110.0,73.0,95.0,118.0,73.0,131.0,127.0,156.0,282.0,179.0,209.0,264.0,512.0,168.0,153.0,44.0


# Appendix 

**`table_number=1` All Data from the second table: Credible Fear Cases Subject to Third Country Transit Bar - Interim Final Rule2**

In [29]:
cfr_third_country_bar = attempt_to_collect_and_combine_all_subsheets(
    base_folder, 1, primary_column="From-To", secondary_column="Unnamed: 2"
)
create_final_summary(cfr_third_country_bar, cfr_third_country_bar.columns[0])


 data/raw_source_files/uscis-cfr-semi-monthly-reports/Congressional-Semi-Monthly-Report-01-01-20-to-01-15-21.xlsx
Likley Successfull Extracting Data - Check Results

 data/raw_source_files/uscis-cfr-semi-monthly-reports/Congressional-Semi-Monthly-Report-02-01-20-to-02-15-21.xlsx
Likley Successfull Extracting Data - Check Results

 data/raw_source_files/uscis-cfr-semi-monthly-reports/Congressional-Semi-Monthly-Report-06-01-20-to-06-15-21.xlsx
Likley Successfull Extracting Data - Check Results

 data/raw_source_files/uscis-cfr-semi-monthly-reports/Congressional-Semi-Monthly-Report-04-01-20-to-04-15-21.xlsx
Likley Successfull Extracting Data - Check Results

 data/raw_source_files/uscis-cfr-semi-monthly-reports/Congressional-Semi-Monthly-Report-05-01-20-to-05-15-21.xlsx
Likley Successfull Extracting Data - Check Results

 data/raw_source_files/uscis-cfr-semi-monthly-reports/Congressional-Semi-Monthly-Report-10-01-20-to-10-15-21.xlsx
Likley Successfull Extracting Data - Check Results

 da

,From-To-Unnamed: 2,01/2020,02/2020,03/2020,04/2020,05/2020,06/2020,07/2020,08/2020,09/2020,10/2020,11/2020,12/2020,01/2021,02/2021,03/2021,04/2021,05/2021,06/2021,07/2021,08/2021,09/2021,10/2021
0,Subject to Bar under 8 CFR 208.13(c)(4) - No Exception Established-Case Receipts,2911.0,2583.0,2140.0,502.0,307.0,192.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Subject to Bar under 8 CFR 208.13(c)(4) - No Exception Established-All Decisions,2126.0,2763.0,2629.0,1360.0,503.0,360.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,Subject to Bar under 8 CFR 208.13(c)(4) - No Exception Established-Fear Established (Y),667.0,1054.0,843.0,610.0,217.0,182.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,Subject to Bar under 8 CFR 208.13(c)(4) - No Exception Established-Fear Not Established (N),1126.0,1335.0,1370.0,675.0,248.0,152.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Subject to Bar under 8 CFR 208.13(c)(4) - No Exception Established-Administratively Closed,333.0,374.0,416.0,75.0,38.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Subject to Bar under 8 CFR 208.13(c)(4) - Exception Established-Case Receipts,49.0,58.0,73.0,25.0,13.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,Subject to Bar under 8 CFR 208.13(c)(4) - Exception Established-All Decisions,35.0,37.0,61.0,68.0,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
7,Subject to Bar under 8 CFR 208.13(c)(4) - Exception Established-Fear Established (Y),31.0,32.0,49.0,56.0,20.0,23.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
8,Subject to Bar under 8 CFR 208.13(c)(4) - Exception Established-Fear Not Established (N),4.0,5.0,12.0,11.0,5.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,Subject to Bar under 8 CFR 208.13(c)(4) - Exception Established-Administratively Closed,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**`table_number=2` All Data from the third table: Credible Fear Cases at Family Residential Centers**

In [30]:
cfr_rfc = attempt_to_collect_and_combine_all_subsheets(
    base_folder, 2, primary_column="From-To"
)
create_final_summary(cfr_rfc, cfr_rfc.columns[0])


 data/raw_source_files/uscis-cfr-semi-monthly-reports/Congressional-Semi-Monthly-Report-01-01-20-to-01-15-21.xlsx
Likley Successfull Extracting Data - Check Results

 data/raw_source_files/uscis-cfr-semi-monthly-reports/Congressional-Semi-Monthly-Report-02-01-20-to-02-15-21.xlsx
Likley Successfull Extracting Data - Check Results

 data/raw_source_files/uscis-cfr-semi-monthly-reports/Congressional-Semi-Monthly-Report-06-01-20-to-06-15-21.xlsx
Likley Successfull Extracting Data - Check Results

 data/raw_source_files/uscis-cfr-semi-monthly-reports/Congressional-Semi-Monthly-Report-04-01-20-to-04-15-21.xlsx
Likley Successfull Extracting Data - Check Results

 data/raw_source_files/uscis-cfr-semi-monthly-reports/Congressional-Semi-Monthly-Report-05-01-20-to-05-15-21.xlsx
Likley Successfull Extracting Data - Check Results

 data/raw_source_files/uscis-cfr-semi-monthly-reports/Congressional-Semi-Monthly-Report-10-01-20-to-10-15-21.xlsx
Likley Successfull Extracting Data - Check Results

 da

,From-To,01/2020,02/2020,03/2020,04/2020,05/2020,06/2020,07/2020,08/2020,09/2020,10/2020,11/2020,12/2020,01/2021,02/2021,03/2021,04/2021,05/2021,06/2021,07/2021,08/2021,09/2021,10/2021
0,Case Receipts,802.0,1234.0,692.0,45.0,60.0,102.0,272.0,180.0,229.0,209.0,183.0,327.0,441.0,385.0,0.0,0.0,0.0,3.0,41.0,1.0,2.0,0.0
1,All Decisions,813.0,875.0,1104.0,171.0,52.0,73.0,253.0,182.0,135.0,334.0,131.0,286.0,371.0,567.0,4.0,2.0,0.0,0.0,26.0,15.0,0.0,0.0
2,Fear Established (Y),268.0,233.0,378.0,56.0,33.0,50.0,125.0,131.0,70.0,204.0,112.0,206.0,283.0,409.0,4.0,2.0,0.0,0.0,9.0,11.0,0.0,0.0
3,Fear Not Established (N),438.0,549.0,603.0,109.0,7.0,19.0,95.0,48.0,46.0,110.0,9.0,15.0,23.0,75.0,0.0,0.0,0.0,0.0,13.0,5.0,0.0,0.0
4,Administratively Closed,107.0,93.0,123.0,6.0,12.0,4.0,33.0,3.0,19.0,20.0,10.0,65.0,65.0,83.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0


**`table_number=3` All Data from the fourth table: All Resonable Fear Claims**

In [ ]:
all_reasonable_fear_claims

,From-To,time_label,04/2020,05/2020,06/2020,07/2020,08/2020,09/2020,10/2020,11/2020,12/2020,01/2021,02/2021,03/2021,04/2021,05/2021,06/2021,07/2021,08/2021,09/2021,01/2020,02/2020,03/2020,10/2021
3,Fear Established (Y),01-01-20,56.0,59.0,42.0,49.0,36.0,38.0,49,61,86,44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,89.0,139.0,122.0,NaN
5,Administratively Closed,01-01-20,212.0,186.0,158.0,155.0,132.0,147.0,170,131,136,42,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,374.0,365.0,349.0,NaN
4,Fear Not Established (N),01-01-20,245.0,166.0,188.0,173.0,196.0,196.0,183,202,206,108,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,359.0,371.0,402.0,NaN
2,All Decisions,01-01-20,513.0,411.0,388.0,377.0,364.0,381.0,402,394,428,194,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,822.0,875.0,873.0,NaN
1,Case Receipts,01-01-20,623.0,440.0,439.0,453.0,414.0,406.0,464,448,429,245,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1027.0,1013.0,1266.0,NaN
5,Administratively Closed,02-01-20,212.0,186.0,158.0,155.0,132.0,147.0,170,131,136,82,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,365.0,349.0,NaN
4,Fear Not Established (N),02-01-20,245.0,166.0,188.0,173.0,196.0,196.0,183,202,206,210,124.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,371.0,402.0,NaN
3,Fear Established (Y),02-01-20,56.0,59.0,42.0,49.0,36.0,38.0,49,61,86,93,43.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,139.0,122.0,NaN
2,All Decisions,02-01-20,513.0,411.0,388.0,377.0,364.0,381.0,402,394,428,385,202.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,875.0,873.0,NaN
1,Case Receipts,02-01-20,623.0,440.0,439.0,453.0,414.0,406.0,464,448,429,482,237.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1013.0,1267.0,NaN


<IPython.core.display.Javascript object>

In [ ]:
all_reasonable_fear_claims = attempt_to_collect_and_combine_all_subsheets(
    base_folder, 3, primary_column="From-To"
)
create_final_summary(all_reasonable_fear_claims, all_reasonable_fear_claims.columns[0])

<IPython.core.display.Javascript object>


 cfr/Congressional-Semi-Monthly-Report-04-01-20-to-04-15-21.xlsx


<IPython.core.display.Javascript object>

Likley Successfull Extracting Data - Check Results

 cfr/Congressional-Semi-Monthly-Report-09-01-20-to-09-15-21.xlsx


<IPython.core.display.Javascript object>

...

Likley Successfull Extracting Data - Check Results

 cfr/Congressional-Semi-Monthly-Report-08-01-20-to-08-15-21.xlsx


<IPython.core.display.Javascript object>

ERROR - Could not detect equal number of beginnings and ends


,From-To,01/2020,02/2020,03/2020,04/2020,05/2020,06/2020,07/2020,08/2020,09/2020,10/2020,11/2020,12/2020,01/2021,02/2021,03/2021,04/2021,05/2021,06/2021,07/2021,08/2021,09/2021,10/2021
0,Fear Established (Y),89.0,139.0,122.0,56.0,59.0,42.0,49.0,36.0,38.0,49.0,61.0,87.0,93.0,81.0,86.0,73.0,90.0,100.0,150.0,108.0,98.0,63.0
1,Administratively Closed,374.0,365.0,349.0,212.0,186.0,158.0,155.0,132.0,147.0,170.0,131.0,136.0,83.0,62.0,59.0,59.0,48.0,83.0,75.0,75.0,81.0,39.0
2,Fear Not Established (N),359.0,371.0,402.0,245.0,166.0,188.0,173.0,196.0,196.0,183.0,202.0,205.0,210.0,220.0,182.0,171.0,159.0,198.0,231.0,192.0,215.0,91.0
3,All Decisions,822.0,875.0,873.0,513.0,411.0,388.0,377.0,364.0,381.0,402.0,394.0,428.0,386.0,363.0,327.0,303.0,297.0,380.0,454.0,375.0,388.0,192.0
4,Case Receipts,1027.0,1013.0,1267.0,623.0,440.0,437.0,453.0,414.0,406.0,464.0,448.0,430.0,482.0,363.0,374.0,369.0,335.0,496.0,447.0,395.0,503.0,186.0


<IPython.core.display.Javascript object>